In [1]:
import pandas as pd
import re
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import ast
import spacy

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
stemmer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
target = pd.read_csv('data/ner_tags_2015_ver_1.csv')


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
n_cluster = 15
cluster_vector = []
cluster_feature = []

for i in range(n_cluster):
    cluster_idx = target.cluster == i

    ne_set = list({ne for ne_list in target[cluster_idx]['ne'] for ne in ast.literal_eval(ne_list)})
    ne_id = [id for id in range(len(ne_set))]
    vocap = dict(zip(ne_set, ne_id))
    
    vectorizer = CountVectorizer()
    vectorizer.vocabulary = vocap
    
    X_count = vectorizer.transform(target[cluster_idx].body).toarray()
    cluster_vector.append(X_count)
    cluster_feature.append(vectorizer.get_feature_names())

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
cluster_vector_tfidf = [TfidfTransformer().fit_transform(x).toarray() for x in cluster_vector]
df_tfidf = [pd.DataFrame(cluster_vector_tfidf[i], columns= cluster_feature[i])for i in range(n_cluster)]

In [13]:
def is_proper(string):
    for c in string:
        if c.isupper():
            return True
    return False

In [14]:
n_cluster = 15
for i in range(n_cluster):
    # remove word 'cluster'
    tf_idf_ne = df_tfidf[i].sum(axis=0).sort_values(ascending=False).keys()
    print("==============topic candidate: {}==============".format(i))
    result = [i for i in list(tf_idf_ne) if is_proper(i)]
    print(result[:10])
    # print(list(tf_idf_ne)[:15])
print("=============================================")

==============topic candidate: 0==============
['Marie Harf', 'Global Times', 'Malays Chinese', 'David Cameron', 'Japan Kyodo News', 'North Korea Consensus', 'the Army Air Force', 'RAND Corp', 'Mairead Maguire', 'Hanwha Thales']
==============topic candidate: 1==============
['UAE', 'the World Bank', 'Kim Jae', 'Park Dae', 'Pyon', 'UNESCO World Heritage Sites', 'Asia Women Fund', 'the United Nations Security Council', 'Lee Byung', 'Korea']
==============topic candidate: 2==============
['Uijeongbu', 'the Seoul International Women Film Festival Park', 'Navy', 'the Ministry of Environment', 'al Qaida', 'the Ministry of Government Administration', 'jong Hong', 'Stephen Kim', 'Dae', 'the Seoul Northern District Court']
==============topic candidate: 3==============
['Jonathan Powell', 'AIPS', 'Renault SA', 'the Republic of Korea', 'Peninsula Abe', 'Seungjengwon Ilgi', 'the Academy of Fine Arts', 'Carlos Ghosn', 'the Sky Team', 'Washington Harris']
==============topic candidate: 4==========